In [5]:
import numpy as np 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import scipy.signal as sig
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from sklearn.model_selection import train_test_split

In [14]:
def add_awgn_noise(signal,SNR_dB):
    """  Adds AWGN noise vector to signal 
         to generate a resulting signal vector y of specified SNR in dB
    """
    L=len(signal)
    SNR = 10**(SNR_dB/10.0) #SNR to linear scale
    Esym=np.sum(np.square(np.abs(signal)))/L #Calculate actual symbol energy
    N0=Esym/SNR; #Find the noise spectral density
    if(isinstance(signal[0], complex)):
        noiseSigma=np.sqrt(N0/2.0)#Standard deviation for AWGN Noise when x is complex
        n = noiseSigma*(np.random.randn(1,L)+1j*np.random.randn(1,L))#computed noise 
    else:
        noiseSigma = np.sqrt(N0);#Standard deviation for AWGN Noise when x is real
        n = noiseSigma*np.random.randn(1,L)#computed noise
    y = signal + n #received signal
    
    return y.flatten()

def generate_preamble_datasets(dataset_size, preamble_length, channel_length, SNR):
        
    preambles = np.random.randint(0, 2, (dataset_size, preamble_length))
    convolved_preambles = []
    channels = []
    for i in range(dataset_size):
        channel_taps = np.random.uniform(0,1,channel_length)
        if sum(channel_taps)>=1:
            channel_taps = channel_taps / sum(channel_taps)
            
        channels.append(channel_taps)
        preamble_conv = add_awgn_noise(sig.convolve(preambles[i], channel_taps, mode='same'), SNR)
        convolved_preambles.append(preamble_conv)

    X = np.hstack([preambles, np.array(convolved_preambles)])
    Y = np.array(channels)
    return X, Y

X, Y = generate_preamble_datasets(10000, 50, 2, 10)
X = X.reshape((X.shape[0], 1, 10, 10))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
def baseline_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (1, 2), input_shape=(1, 10, 10), activation='relu'))
    model.add(MaxPooling2D(pool_size=(1, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='relu'))
    # Compile model
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    return model
print(X_train.shape, Y_train.shape)

(8000, 1, 10, 10) (8000, 2)


In [16]:
model = baseline_model()
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 1s - loss: 0.1391 - mean_squared_error: 0.1391 - val_loss: 0.1373 - val_mean_squared_error: 0.1373
Epoch 2/100
8000/8000 [==============================] - 0s - loss: 0.0525 - mean_squared_error: 0.0525 - val_loss: 0.0384 - val_mean_squared_error: 0.0384
Epoch 3/100
8000/8000 [==============================] - 0s - loss: 0.0373 - mean_squared_error: 0.0373 - val_loss: 0.0323 - val_mean_squared_error: 0.0323
Epoch 4/100
8000/8000 [==============================] - 0s - loss: 0.0314 - mean_squared_error: 0.0314 - val_loss: 0.0282 - val_mean_squared_error: 0.0282
Epoch 5/100
8000/8000 [==============================] - 0s - loss: 0.0264 - mean_squared_error: 0.0264 - val_loss: 0.0222 - val_mean_squared_error: 0.0222
Epoch 6/100
8000/8000 [==============================] - 0s - loss: 0.0232 - mean_squared_error: 0.0232 - val_loss: 0.0198 - val_mean_squared_error: 0.0198
Epoch 7/100
8000

8000/8000 [==============================] - 0s - loss: 0.0094 - mean_squared_error: 0.0094 - val_loss: 0.0100 - val_mean_squared_error: 0.0100
Epoch 53/100
8000/8000 [==============================] - 0s - loss: 0.0093 - mean_squared_error: 0.0093 - val_loss: 0.0106 - val_mean_squared_error: 0.0106
Epoch 54/100
8000/8000 [==============================] - 0s - loss: 0.0093 - mean_squared_error: 0.0093 - val_loss: 0.0105 - val_mean_squared_error: 0.0105
Epoch 55/100
8000/8000 [==============================] - 0s - loss: 0.0092 - mean_squared_error: 0.0092 - val_loss: 0.0101 - val_mean_squared_error: 0.0101
Epoch 56/100
8000/8000 [==============================] - 0s - loss: 0.0093 - mean_squared_error: 0.0093 - val_loss: 0.0103 - val_mean_squared_error: 0.0103
Epoch 57/100
8000/8000 [==============================] - 0s - loss: 0.0093 - mean_squared_error: 0.0093 - val_loss: 0.0104 - val_mean_squared_error: 0.0104
Epoch 58/100
8000/8000 [==============================] - 0s - loss: 0.